# Occupancy Grid Maps


## Mapping Task
Compute the most likely map given the sensor data
$$m^* = argmax_m P(m|u_1, z_1,...,u_t, z_t)$$


# Exercise 3

## Occupancy grid mapping

The aim of this assignment is to implement a 2D occupancy grid mapping algorithm as described in the lecture. We provide data recorded by a robot using a 2D laser scanner in an indoor environment. 

The raw data is arranged in the following manner:

- `poses.data`: contains the **known** robot poses as it moves around the environment. Each line corresponds to the pose of the robot at a time instant

- `ranges.data`: contains 2d laser range measurements recorded by the robot. Each line corresponds to the range measurements taken from the corresponding robot poses.

---

### Log odds to probability

The occupancy calculations are usually performed using the log odds. We will need to convert between log odds and probabilities in this exercise. Implement the `logodds2prob` and `prob2logodds` functions. Verify that the conversion functions are correct for some test input. **[2.0]**

In [5]:
#pip install bresenham

Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
from src.ex3 import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Computes the probability from log-odds form
def logodds2prob(l):
    return 1-(1/(1+np.exp(l)))

# Computes the log-odds from probability
def prob2logodds(p):
    return np.log(p/(1-p))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
### Inverse sensor model

The occupancy grid mapping relies on as inverse sensor model to update the occupancy of a cell in the grid. For this exercise, we consider a simple sensor model, where the occupancy of cell $m_i$ is $P_{occ}$ if the measurement $z_t$ falls in that cell, otherwise the occupancy is $P_{free}$. 

However, if the cell $m_i$ is not in the perception field, the occupancy will be $P_{prior}$.

Use the  following probabilities in the implementation:
$$P_{occ} = 0.9, P_{free} = 0.35, P_{prior} = 0.5 $$

Implement the inverse sensor model function and verify that it is correct for some test input. **[2.0]**

In [70]:
cell = [1, 1]
endpoint = [7,9]
prob_occ = 0.9
prob_free = 0.35

def inv_sensor_model(cell, endpoint, prob_occ, prob_free):
    line = bresenham(cell[0], cell[1], endpoint[0], endpoint[1])
    p_values = [0] * len(line)
    for i in range(len(line) - 1):
        p_values[i] = prob_free
    
    p_values[-1] = prob_occ
    inv_sensor_model = np.zeros((len(line),3))
    for i, l in enumerate(line):
        inv_sensor_model[i] = [*l, p_values[i]]

    return inv_sensor_model

test = inv_sensor_model(cell, endpoint, prob_occ, prob_free)
print(test)


[[1.   1.   0.35]
 [2.   2.   0.35]
 [3.   3.   0.35]
 [3.   4.   0.35]
 [4.   5.   0.35]
 [5.   6.   0.35]
 [6.   7.   0.35]
 [6.   8.   0.35]
 [7.   9.   0.9 ]]


---
### Occupancy grid mapping algorithm:

Implement the occupancy grid mapping algorithm as described in lecture slides. 

We provide some functions to make the implemntation easier.

- `ranges2cells` converts the raw measurements acquired by the robot (*ranges_raw*) into the correspoding cells of the gridmap.
- `poses2cells` converts the raw poses of the robot (*poses_raw*) into the correspoding cells of the gridmap.
- `bresenham` returns all the cells along a straigh line between two points in the gridmap.

Other parameters you may need in the implemntation are provided below.

Implement the `grid_mapping_with_known_poses` functionn and compute the resulting occupancy grid map using all the measurements provided in the data.

Plot the final occupancy gridmap using the `plot_gridmap` function. **[6.0]**


In [100]:
from src.ex3 import *
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

map_size = 100
map_res = 0.25

prior = 0.50
prob_occ = 0.90
prob_free = 0.35

# load data
ranges_raw = np.loadtxt("data/grid_mapping/ranges.data", delimiter=',', dtype='float')
poses_raw = np.loadtxt("data/grid_mapping/poses.data", delimiter=',', dtype='float')

# initialize gridmap
occ_gridmap = init_gridmap(map_size, map_res)+prior
plot_gridmap(occ_gridmap)

updated_gridmap = grid_mapping_with_known_poses(ranges_raw, poses_raw, occ_gridmap, map_res, prob_occ, prob_free, prior)

plot_gridmap(updated_gridmap)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Pose: 10
Pose: 20
Pose: 30
Pose: 40
Pose: 50
Pose: 60
Pose: 70
Pose: 80
Pose: 90
Pose: 100
Pose: 110
Pose: 120
Pose: 130
